In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
dat_files = [x for x in os.listdir() if x.endswith('.dat') and not x.startswith('.')]

for dat_file in dat_files:
    time, flux, flux_err = np.loadtxt(dat_file, unpack=True)
    splited_dat_name = dat_file.split('.')
    rename = '.'.join([splited_dat_name[0], splited_dat_name[2], splited_dat_name[1],'csv'])
    np.savetxt(rename, np.array([time, flux, flux_err]).T, delimiter=',', header='#time,flux,flux_err', comments='', fmt='%.8f')

In [2]:
import numpy as np
import pandas as pd
import glob
parameter = {}
parameter['b_rr'] = 0.0488
rr = parameter['b_rr']
ar = 18.79
t0 = 2458525.04109
period = 8.46308

b_rsuma = (1+rr)/ar
parameter['b_rsuma'] = b_rsuma
parameter['b_cosi'] = 0
parameter['b_epoch'] = t0
parameter['b_period'] = period
parameter['b_K'] = 0.396
parameter['b_f_c'] = 0
parameter['b_f_s'] = 0
parameter['host_lambda'] = 0
parameter['host_vsini'] = 8.5


In [3]:

def make_params(parameter, inst_phot, inst_rv, inst_rm):
    rr = parameter['b_rr']; rsuma = parameter['b_rsuma']; cosi = parameter['b_cosi']; epoch = parameter['b_epoch']; period = parameter['b_period']; k = parameter['b_K']; f_c = parameter['b_f_c']; f_s = parameter['b_f_s']
    ### print

    with open('params.csv', 'w') as f:
        print('#name,value,fit,bounds,label,unit,coupled_with', file=f)
        print('#baselines,,,,,,', file=f)

        print('b_rr,'+str(rr)+',1,uniform 0 1,$R_b / R_\star$,,', file=f)
        print('b_rsuma,'+str(rsuma)+',1,uniform 0 1,$(R_\star + R_b) / a_b$,,', file=f)
        print('b_cosi,'+str(cosi)+',1,uniform 0 1,$\cos{i_b}$,,', file=f)
        print('b_epoch,'+str(epoch)+',1,uniform '+str(epoch-1)+' '+str(epoch+1)+',$t_0$,HJDUTC,', file=f)
        print('b_period,'+str(period)+',1,uniform 0 100,$P_b$,d,', file=f)
        print('b_K,'+str(k)+',1,uniform 0 2,$K_b$,km/s,', file=f)
        print('b_f_c,'+str(f_c)+',1,uniform -1.0 1.0,$\sqrt{e_b} \cos{\omega_b}$,,', file=f)
        print('b_f_s,'+str(f_s)+',1,uniform -1.0 1.0,$\sqrt{e_b} \sin{\omega_b}$,,', file=f)


        ### dilution
        print('#dilution per instrument,,,,,', file=f)
        # for ele in inst_phot:
        #     print('dil_'+ele+','+str(parameter['dil_'+ele])+',1,uniform 0 1,$dilution_\mathrm{'+ele+'}$,,')
        ### ld
        print('#limb darkening coefficients per instrument,,,,,', file=f)
        for ele in inst_rm:
            # band = ele.split('_')[-1]
            # print(band)
            print('host_ldc_q1_'+ele+','+str(parameter['host_ldc_q1_'+ele])+',1,uniform 0 1,$q_{1;\mathrm{'+ele+'}}$,,', file=f)
            print('host_ldc_q2_'+ele+','+str(parameter['host_ldc_q2_'+ele])+',1,uniform 0 1,$q_{2;\mathrm{'+ele+'}}$,,', file=f)
            
            
            data = np.loadtxt(ele+'.csv',delimiter=',')
            rv = data[:,1]
            med_rv = np.median(rv)
            
            ## baseline
            # print('baseline_gp_offset_rv_'+ele+',0,1,uniform '+str(med_rv-1)+' '+str(med_rv+1)+',$\mathrm{gp offset ('+ele+')}$,,', file=f)
            # print('baseline_gp_matern32_lnsigma_rv_'+ele+',-5,1,uniform -15 0,$\mathrm{gp ln sigma ('+ele+')}$,,', file=f)
            # print('baseline_gp_matern32_lnrho_rv_'+ele+',0,1,uniform -1 15,$\mathrm{gp ln rho ('+ele+')}$,,', file=f)
# baseline_gp_offset_flux_Leonardo,0,1,uniform -0.01 0.01,$\mathrm{gp ln sigma (Leonardo)}$,
# baseline_gp_matern32_lnsigma_flux_Leonardo,-5,1,uniform -15 0,$\mathrm{gp ln sigma (Leonardo)}$,
# baseline_gp_matern32_lnrho_flux_Leonardo,0,1,uniform -1 15,$\mathrm{gp ln rho (Leonardo)}$,

        

        bands = np.asarray([x.split('.')[1] for x in inst_phot])
        unique_bands = np.unique(bands)
            

        for band in unique_bands:
            idx = np.where(bands == band)[0]
            for count_band,i in enumerate(idx):
                if count_band>0:
                    print('host_ldc_q1_'+inst_phot[i]+',,,,,,'+'host_ldc_q1_'+inst_phot[idx[0]], file=f)
                    print('host_ldc_q2_'+inst_phot[i]+',,,,,,'+'host_ldc_q2_'+inst_phot[idx[0]], file=f)
                else:
                    print('host_ldc_q1_'+inst_phot[i]+','+str(parameter['host_ldc_q1_'+inst_phot[i]])+',1,uniform 0 1,$q_{1;\mathrm{'+inst_phot[i]+'}}$,,', file=f)
                    print('host_ldc_q2_'+inst_phot[i]+','+str(parameter['host_ldc_q2_'+inst_phot[i]])+',1,uniform 0 1,$q_{2;\mathrm{'+inst_phot[i]+'}}$,,', file=f)
        # ldc_q1_HAT-P-1_56_TESS,,,,,,
        print('#errors per instrument,', file=f)
        
        no_tess_inst_phot = np.asarray([x for x in inst_phot if 'TESS' not in x])
        tess_inst_phot = np.asarray([x for x in inst_phot if 'TESS' in x])
        print(inst_phot)
        for ele in no_tess_inst_phot:
            print('ln_err_flux_'+ele+','+str(parameter['ln_err_flux_'+ele])+',1,uniform -15 0.0,$\ln{\sigma_\mathrm{flux; '+ele+'}}$,$\ln{ \mathrm{mag} }$,', file=f)
            
        for i,ele in enumerate(tess_inst_phot):
            if i <1:
                print('ln_err_flux_'+ele+','+str(parameter['ln_err_flux_'+ele])+',1,uniform -15 0.0,$\ln{\sigma_\mathrm{flux; '+ele+'}}$,$\ln{ \mathrm{mag} }$,', file=f)
            else:
                print('ln_err_flux_'+ele+',,,,,,ln_err_flux_'+tess_inst_phot[0], file=f)            
            
            
            
        for ele in np.append(inst_rm, inst_rv):
            print('ln_jitter_rv_'+ele+','+str(parameter['ln_jitter_rv_'+ele])+',1,uniform -15 0.0,$\ln{\sigma_\mathrm{rv; '+ele+'}}$,$\ln{ \mathrm{km/s} }$,', file=f)
        print('### lambda and vsini', file=f)
        print('host_lambda,'+str(parameter['host_lambda'])+',1,uniform -360 360,$\lambda_\mathrm{host}$,,', file=f)
        print('host_vsini,'+str(parameter['host_vsini'])+',1,normal '+str(parameter['host_vsini'])+' 1.5,$v\sin{i}_\mathrm{host}$,$\mathrm{km/s}$,', file=f)
              #0 100,$v\sin{i}_\mathrm{host}$,$\mathrm{km/s}$,', file=f)
# make_params(parameter)
### setting 

# name,value
# ##############################################################################,
# General settings,
# ##############################################################################,

def make_settings(inst_phot, inst_rv, inst_rm):
    with open('settings.csv','w') as f:

        print('names,values',file=f)
        print('##############################################################################,',file=f)
        print('### General settings',file=f)
        print('##############################################################################,',file=f)

        # companions_phot,b
        # companions_rv,b
        # inst_phot,Wise_I_4069_LC_HJDUTC merged_GUNNZ_LC_HJDUTC merged_Sloanz_LC_HJDUTC
        # inst_rv,HDS_RV_HJDUTC HDS_RM_HJDUTC HIRES_RM_HJDUTC
        print('companions_phot,b',file=f)
        print('companions_rv,b',file=f)
        print('inst_phot,',end='',file=f)
        for ele in inst_phot:
            print(ele,end=' ',file=f)
        print(file=f)
        print('inst_rv,',end='',file=f)
        for ele in np.append(inst_rm, inst_rv):
            print(ele,end=' ',file=f)
        print(file=f)
        print('''###############################################################################,
# Fit performance settings,
###############################################################################,
multiprocess,True
multiprocess_cores,all
fast_fit,True
fast_fit_width,0.3333333333333333
secondary_eclipse,False
phase_curve,False
shift_epoch,True
inst_for_b_epoch,all''',file=f)
        print('''###############################################################################,
# MCMC settings,
###############################################################################,
mcmc_nwalkers,100
mcmc_total_steps,10000
mcmc_burn_steps,1000
#mcmc_thin_by,10
#mcmc_pre_run_loops,2
#mcmc_pre_run_steps,1000
mcmc_total_steps_over_max_tau,50
de_steps,5000
###############################################################################,
# Nested Sampling settings,
###############################################################################,
ns_modus,dynamic
ns_nlive,500
ns_bound,single
ns_sample,rwalk
ns_tol,0.01''',file=f)
        print('''###############################################################################,
# Limb darkening law per object and instrument,
# if 'lin' one corresponding parameter called 'ldc_q1_inst' has to be given in params.csv,
# if 'quad' two corresponding parameter called 'ldc_q1_inst' and 'ldc_q2_inst' have to be given in params.csv,
# if 'sing' three corresponding parameter called 'ldc_q1_inst'; 'ldc_q2_inst' and 'ldc_q3_inst' have to be given in params.csv,
 ###############################################################################,''',file=f)
        for ele in np.append(inst_phot, inst_rm):
            print('host_ld_law_'+ele+',quad',file=f)
        print('''###############################################################################,
# Baseline settings per instrument,
# baseline params per instrument: sample_offset / sample_linear / sample_GP / hybrid_offset / hybrid_poly_1 / hybrid_poly_2 / hybrid_poly_3 / hybrid_pol_4 / hybrid_spline / hybrid_GP,
# if 'sample_offset' one corresponding parameter called 'baseline_offset_key_inst' has to be given in params.csv,
# if 'sample_linear' two corresponding parameters called 'baseline_a_key_inst' and 'baseline_b_key_inst' have to be given in params.csv,
# if 'sample_GP' two corresponding parameters called 'baseline_gp1_key_inst' and 'baselie_gp2_key_inst' have to be given in params.csv,
###############################################################################,''',file=f)



        for ele in inst_phot:
            baseline = 'hybrid_offset'
            if "TESS" in ele:
                baseline = 'hybrid_offset'
            print('baseline_flux_'+ele+','+baseline,file=f)
        for ele in inst_rv:
            print('baseline_rv_'+ele+',hybrid_offset',file=f)

        for ele in inst_rm:
            print('baseline_rv_'+ele+',hybrid_offset',file=f)

        print('''###############################################################################,
# Error settings per instrument,
# errors (overall scaling) per instrument: sample / hybrid,
# if 'sample' one corresponding parameter called 'ln_err_key_inst' (photometry) or 'ln_jitter_key_inst' (RV) has to be given in params.csv,
###############################################################################,''',file=f)


        for ele in inst_phot:
            print('error_flux_'+ele+',sample',file=f)
        for ele in inst_rv:
            print('error_rv_'+ele+',sample',file=f)
        print('''# TTVs,
###############################################################################,
fit_ttvs,False
###############################################################################,
# Flux weighted RVs per object and instrument,
# ("Yes" for Rossiter-McLaughlin effect),
###############################################################################,''',file=f)

        for ele in inst_rm:
            print('b_flux_weighted_'+ele+',True',file=f)

        print('''###############################################################################,''')
        for key in parameter.keys():
            if 'exp' in key:
                print(key,parameter[key],file=f,sep=',')




# host_ldc_q1_NEID_RM


inst_phot = [x.strip('.csv') for x in np.sort(np.asarray(glob.glob('*.csv'))) if 'RM' not in x and 'RV' not in x and 'param' not in x and 'settings' not in x]

# inst_phot = [inst_phot[0], inst_phot[-7]]

inst_rm = [x.strip('.csv') for x in np.sort(np.asarray(glob.glob('*.RM.csv')))]
inst_rv = [x.strip('.csv') for x in np.sort(np.asarray(glob.glob('*.RV.csv')))]


for inst in inst_phot+inst_rm:
    parameter['host_ldc_q1_'+inst] = 0.5
    parameter['host_ldc_q2_'+inst] = 0.5

for inst in inst_phot+inst_rm:
    parameter['ln_err_flux_'+inst] = -7

for inst in inst_rm+inst_rv:
    parameter['ln_jitter_rv_'+inst] = -7
    
    
for inst in inst_rm:
    data = np.loadtxt(inst+'.csv',delimiter=',')
    t_exp = np.median(np.diff(data[:,0]))
    t_exp_sec = t_exp*24*3600
    if t_exp_sec > 150:
        parameter['t_exp_'+inst] = t_exp
        parameter['t_exp_n_int_'+inst] = 10


make_params(parameter, inst_phot, inst_rv, inst_rm)
make_settings(inst_phot, inst_rv, inst_rm)


['TESS.LC']
###############################################################################,


In [ ]:
import numpy as np
def _estimate_vmac_doyle2014(teff, logg, feh):
    """
    Estimate Macroturbulence velocity (Vmac) by using an empirical relation
    considering the effective temperature, surface gravity and metallicity.

    The relation was constructed by Doyle et al. (2014), which is only valid
    for the Teff range 5200 to 6400 K, and the log g range 4.0 to 4.6 dex.
    """
    t0 = 5777
    g0 = 4.44

    if logg >= 3.5:
        if teff >= 5000:
            # main sequence and subgiants (RGB)
            vmac = 3.21 + 2.33e-3*(teff-t0) + 2e-6*(teff-t0)**2 - 2*(logg-g0)
        else:
            # main sequence
            vmac = 3.21 + 2.33e-3*(teff-t0) + 2e-6*(teff-t0)**2 - 2*(logg-g0)
    else:
        # Out of the calibrated limits
        vmac = 0.

    return vmac

def _estimate_vmac_ges(teff, logg, feh):
    """
    Estimate Microturbulence velocity (Vmic) by using an empirical relation
    considering the effective temperature, surface gravity and metallicity.

    The relation was constructed by Maria Bergemann for the Gaia ESO Survey.
    """
    t0 = 5500
    g0 = 4.0

    if logg >= 3.5:
        if teff >= 5000:
            # main sequence and subgiants (RGB)
            vmac = 3*(1.15 + 7e-4*(teff-t0) + 1.2e-6*(teff-t0)**2 - 0.13*(logg-g0) + 0.13*(logg-g0)**2 - 0.37*feh - 0.07*feh**2)
        else:
            # main sequence
            vmac = 3*(1.15 + 2e-4*(teff-t0) + 3.95e-7*(teff-t0)**2 - 0.13*(logg-g0) + 0.13*(logg-g0)**2)
    else:
        # giants (RGB/AGB)
        vmac = 3*(1.15 + 2.2e-5*(teff-t0) - 0.5e-7*(teff-t0)**2 - 0.1*(logg-g0) + 0.04*(logg-g0)**2 - 0.37*feh - 0.07*feh**2)

    return vmac

def estimate_vmac(teff, logg, feh, relation='GES'):
    """
    Estimate Microturbulence velocity (Vmic) by using an empirical relation
    considering the effective temperature, surface gravity and metallicity.

    By default, the selected relation was constructed by Maria Bergemann
    for the Gaia ESO Survey. Alternatively, "relation='Doyle2014'" implements
    a relation for dwrafs (Doyle et al, 2014).
    """
    if relation == 'Doyle2014':
        vmac = _estimate_vmac_doyle2014(teff, logg, feh)
    else:
        vmac = _estimate_vmac_ges(teff, logg, feh)
    vmac = float("%.2f" % vmac)
    return vmac

### vmic
def _estimate_vmic_ges(teff, logg, feh):
    """
    Estimate Microturbulence velocity (Vmic) by using an empirical relation
    considering the effective temperature, surface gravity and metallicity.

    The relation was constructed based on the UVES Gaia ESO Survey iDR1 data,
    results for the benchmark stars (Jofre et al. 2013),
    and globular cluster data from external literature sources.

    Source: http://great.ast.cam.ac.uk/GESwiki/GesWg/GesWg11/Microturbulence
    """
    t0 = 5500
    g0 = 4.0

    if logg >= 3.5:
        if teff >= 5000:
            # main sequence and subgiants (RGB)
            vmic = 1.05 + 2.51e-4*(teff-t0) + 1.5e-7*(teff-t0)**2 - 0.14*(logg-g0) - 0.05e-1*(logg-g0)**2 + 0.05*feh + 0.01*feh**2
        else:
            # main sequence
            vmic = 1.05 + 2.51e-4*(5000-t0) + 1.5e-7*(5000-t0)**2 - 0.14*(logg-g0) - 0.05e-1*(logg-g0)**2 + 0.05*feh + 0.01*feh**2
    else:
        # giants (RGB/AGB)
        vmic = 1.25 + 4.01e-4*(teff-t0) + 3.1e-7*(teff-t0)**2 - 0.14*(logg-g0) - 0.05e-1*(logg-g0)**2 + 0.05*feh + 0.01*feh**2
    vmic = float("%.2f" % vmic)
    return vmic

def _estimate_vmic_Bruntt2010(teff, logg, feh):
    # https://ui.adsabs.harvard.edu/abs/2010MNRAS.405.1907B/abstract
    t0 = 5700
    g0 = 4.0

    if logg < 4 or teff < 5000 or teff > 6500:
        return np.nan

    vmic = 1.01 + 4.5610e-4*(teff-t0) + 2.75e-7*(teff-t0)**2
    
    vmic = float("%.2f" % vmic)
    return vmic

def estimate_vmic(teff, logg, feh, relation='GES'):
    """
    Estimate Microturbulence velocity (Vmic) by using an empirical relation
    considering the effective temperature, surface gravity and metallicity.

    By default, the selected relation was constructed by Maria Bergemann
    for the Gaia ESO Survey. Alternatively, "relation='Doyle2014'" implements
    a relation for dwrafs (Doyle et al, 2014).
    """
    if relation == 'Bruntt2010':
        vmac = _estimate_vmic_Bruntt2010(teff, logg, feh)
    else:
        vmac = _estimate_vmic_ges(teff, logg, feh)
    vmac = float("%.2f" % vmac)
    return vmac


teff = 5500
logg = 4.5
feh = 0.0

#vmac = estimate_vmac(teff, logg, feh)
vmic = estimate_vmic(teff, logg, feh)

#print(vmac,vmic)

vmac = estimate_vmac(teff, logg, feh, relation='Doyle2014')
#vmic = estimate_vmic(teff, logg, feh, relation='Bruntt2010')

print(vmac,vmic)